In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from classifiers import *

In [2]:
train_data = pd.read_csv('data/train.csv', sep = '|')
test_data = pd.read_csv('data/test.csv', sep = '|')
print(f'Train set has {train_data.shape[0]} entries and {train_data.shape[1]} features')
# print(f'Test set has {test_data.shape[0]} entries and {test_data.shape[1]} features')
# print(f'attributes: \n {test_data.columns}')
# print(f'types: \n {train_data.dtypes}')

Train set has 1879 entries and 10 features


In [3]:
# encode category feature
trust_encode = pd.get_dummies(train_data['trustLevel'], prefix='trustLevel')
# normalize 
X = train_data.drop(columns=['trustLevel','fraud'])
col = X.columns 
y = train_data['fraud']
# X = StandardScaler().fit_transform(X)
X = trust_encode.join(pd.DataFrame(X, columns=col))
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
evaluate_classification(X, y)
# positive class is not predicted in Random Forest => should check balance labels

Naive Bayes: test core =-9840
K Nerest Neighbors: test core =-1095
Linear SVM: test core =-520
RBF SVM: test core =-520
